In [ ]:
#test comment 

In [1]:
import pandas as pd
import numpy as np
import glob # to find all files in folder
from datetime import datetime
from datetime import date, time
from dateutil.parser import parse
pd.options.mode.chained_assignment = None  # default='warn'

# Advanced Exo from intro to pandas

In [2]:
def load_data():
    sl_files=glob.glob('Data/ebola/sl_data/*.csv')
    guinea_files=glob.glob('Data/ebola/guinea_data/*.csv')
    liberia_files=glob.glob('Data/ebola/liberia_data/*.csv')
    sl = pd.concat((pd.read_csv(file,  parse_dates=['date']) for file in sl_files), ignore_index=True)
    guinea = pd.concat((pd.read_csv(file , parse_dates=['Date']) for file in guinea_files), ignore_index=True)
    liberia = pd.concat((pd.read_csv(file , parse_dates=['Date']) for file in liberia_files), ignore_index=True)
    return (sl, guinea, liberia)

#test github baptiste
    

In [3]:
(sl, guinea, liberia) = load_data()

In [240]:
def select_features(data, var_name, features):
    return data[data[var_name].isin(features)]

In [241]:
# take the relevant variables
sl_relevant = select_features(sl, 'variable', ['new_confirmed', 'death_confirmed'])
guinea_relevant = select_features(guinea, 'Description', ['New cases of confirmed', 'New deaths registered today'])
liberia_relevant = select_features(liberia, 'Variable', ['Total death/s in confirmed cases', 'Total confirmed cases'])

In [242]:
# rename the variables and cols
var_name = 'vars'
sl_relevant.rename(columns={'variable': var_name}, inplace=True)
guinea_relevant.rename(columns={'Description': var_name, 'Date': 'date'}, inplace=True)
liberia_relevant.rename(columns={'Variable': var_name, 'Date': 'date'}, inplace=True)

new_infected = 'new_infected'
new_death= 'new_death'
sl_relevant[var_name][sl_relevant[var_name] == 'new_confirmed'] = new_infected
sl_relevant[var_name][sl_relevant[var_name] == 'death_confirmed'] = new_death

guinea_relevant[var_name][guinea_relevant[var_name] == 'New cases of confirmed'] = new_infected
guinea_relevant[var_name][guinea_relevant[var_name] == 'New deaths registered today'] = new_death

liberia_relevant[var_name][liberia_relevant[var_name] == 'Total confirmed cases'] = new_infected
liberia_relevant[var_name][liberia_relevant[var_name] == 'Total death/s in confirmed cases'] = new_death



In [243]:
# rename the data
sl_clean = sl_relevant
guinea_clean = guinea_relevant
liberia_clean = liberia_relevant

In [244]:
def remove_rows_and_cols_with_only_nan(data):
    return data.dropna(axis=1, how='all').dropna(axis=0, thresh=3)
sl_clean = remove_rows_and_cols_with_only_nan(sl_clean)
guinea_clean = remove_rows_and_cols_with_only_nan(guinea_clean)
liberia_clean = remove_rows_and_cols_with_only_nan(liberia_clean)

In [245]:
# replace all NaNs with 0
sl_clean.fillna(value=0, inplace=True)
guinea_clean.fillna(value=0, inplace=True)
liberia_clean.fillna(value=0, inplace=True)

In [246]:
# make all types numeric
def change_to_numeric(data):
    col_list = list(data.columns)
    col_list.remove('date')
    col_list.remove(var_name)
    data[col_list] = data[col_list].apply(pd.to_numeric)
    
change_to_numeric(sl_clean)
change_to_numeric(guinea_clean)
change_to_numeric(liberia_clean)

In [247]:
#  create a total colon
def add_and_fill_total_col(data, ignore_cols_list):
    col_list = list(data.columns)
    for c in ignore_cols_list:
        col_list.remove(c) 
    data['total'] = data[col_list].sum(axis=1)
add_and_fill_total_col(sl_clean, ['date', var_name, 'National'])
add_and_fill_total_col(guinea_clean, ['date', var_name, 'Totals'])
add_and_fill_total_col(liberia_clean, ['date', var_name, 'National'])

In [248]:
# remove unused cols:
sl_clean = sl_clean[['date', var_name, 'total']]
guinea_clean = guinea_clean[['date', var_name, 'total']]
liberia_clean = liberia_clean[['date', var_name, 'total']]

In [249]:
len(sl_clean)

199

In [250]:
len(guinea_clean)

23

In [251]:
len(liberia_clean)

141

In [252]:
liberia_clean['date']

4      2014-06-16
28     2014-06-16
35     2014-06-17
59     2014-06-17
66     2014-06-22
90     2014-06-22
97     2014-06-24
121    2014-06-24
128    2014-06-25
152    2014-06-25
159    2014-06-28
183    2014-06-28
190    2014-06-29
214    2014-06-29
221    2014-07-01
245    2014-07-01
252    2014-07-02
276    2014-07-02
283    2014-07-03
307    2014-07-03
314    2014-07-07
338    2014-07-07
345    2014-07-08
369    2014-07-08
376    2014-07-10
400    2014-07-10
407    2014-07-13
431    2014-07-13
438    2014-07-17
462    2014-07-17
          ...    
2095   2014-07-26
2102   2014-09-10
2126   2014-09-10
2133   2014-10-03
2157   2014-10-03
2188   2014-10-23
2220   2014-10-24
2252   2014-10-25
2284   2014-10-28
2316   2014-10-29
2348   2014-10-30
2380   2014-10-31
2412   2014-11-02
2444   2014-11-04
2476   2014-11-08
2484   2014-11-14
2508   2014-11-14
2540   2014-11-15
2572   2014-11-19
2636   2014-11-21
2668   2014-11-23
2700   2014-11-24
2732   2014-11-26
2764   2014-11-27
2796   201

In [253]:
#rename data
sl_final = sl_clean
liberia_final = liberia_clean
guinea_final = guinea_clean

In [254]:
liberia_final.head()

,date,vars,total
4,2014-06-16,new_death,4.0
28,2014-06-16,new_infected,12.0
35,2014-06-17,new_death,8.0
59,2014-06-17,new_infected,13.0
66,2014-06-22,new_death,16.0


In [255]:
guinea_final.head()


,date,vars,total
2,2014-08-04,new_infected,4.0
8,2014-08-04,new_death,2.0
44,2014-08-26,new_infected,10.0
76,2014-08-27,new_infected,10.0
108,2014-08-30,new_infected,9.0


In [256]:
sl_final.head()

,date,vars,total
4,2014-08-12,new_infected,11.0
11,2014-08-12,new_death,264.0
33,2014-08-13,new_infected,15.0
40,2014-08-13,new_death,273.0
62,2014-08-14,new_infected,13.0


In [257]:
# create infected and death cols
def create_inf_death_cols(data):
    inf = data[data['vars'] == new_infected]
    inf[new_infected] = inf['total']
    death = data[data['vars'] == new_death]
    death[new_death] = death['total']
    res = data.join(inf[new_infected], how='outer')
    return res.join(death[new_death], how='outer')
sl_final = create_inf_death_cols(sl_final)
liberia_final = create_inf_death_cols(liberia_final)
guinea_final = create_inf_death_cols(guinea_final)

In [258]:
sl_final.head()

,date,vars,total,new_infected,new_death
4,2014-08-12,new_infected,11.0,11.0,NaN
11,2014-08-12,new_death,264.0,NaN,264.0
33,2014-08-13,new_infected,15.0,15.0,NaN
40,2014-08-13,new_death,273.0,NaN,273.0
62,2014-08-14,new_infected,13.0,13.0,NaN


In [259]:
# remove vars & total col
sl_final = sl_final.drop(var_name, 1).drop('total', 1)
liberia_final = liberia_final.drop(var_name, 1).drop('total', 1)
guinea_final = guinea_final.drop(var_name, 1).drop('total', 1)

In [260]:
sl_final.head()

,date,new_infected,new_death
4,2014-08-12,11.0,NaN
11,2014-08-12,NaN,264.0
33,2014-08-13,15.0,NaN
40,2014-08-13,NaN,273.0
62,2014-08-14,13.0,NaN


In [261]:
liberia_final = liberia_final.groupby('date', as_index=False).sum()
sl_final = sl_final.groupby('date', as_index=False).sum()
guinea_final = guinea_final.groupby('date', as_index=False).sum()

In [262]:
# add 'country' col
sl_final['country'] = 'sl'
guinea_final['country'] = 'guinea'
liberia_final['country'] = 'liberia'

In [263]:
guinea_final.head()

,date,new_infected,new_death,country
0,2014-08-04,4.0,2.0,guinea
1,2014-08-26,10.0,NaN,guinea
2,2014-08-27,10.0,NaN,guinea
3,2014-08-30,9.0,NaN,guinea
4,2014-08-31,29.0,NaN,guinea


In [264]:
liberia_final.head()

,date,new_infected,new_death,country
0,2014-06-16,12.0,4.0,liberia
1,2014-06-17,13.0,8.0,liberia
2,2014-06-22,28.0,16.0,liberia
3,2014-06-24,33.0,18.0,liberia
4,2014-06-25,35.0,20.0,liberia


In [265]:
sl_final.head()

,date,new_infected,new_death,country
0,2014-08-12,11.0,264.0,sl
1,2014-08-13,15.0,273.0,sl
2,2014-08-14,13.0,280.0,sl
3,2014-08-15,10.0,287.0,sl
4,2014-08-16,18.0,297.0,sl


In [266]:
final_data = pd.concat([sl_final, guinea_final, liberia_final])
final_data.head()

,date,new_infected,new_death,country
0,2014-08-12,11.0,264.0,sl
1,2014-08-13,15.0,273.0,sl
2,2014-08-14,13.0,280.0,sl
3,2014-08-15,10.0,287.0,sl
4,2014-08-16,18.0,297.0,sl


In [267]:
# final_data['date'] = final_data.index

In [268]:
final_data.set_index(['date', 'country'], inplace=True)

In [270]:
final_data

,,new_infected,new_death
date,country,,
2014-08-12,sl,11.0,264.0
2014-08-13,sl,15.0,273.0
2014-08-14,sl,13.0,280.0
2014-08-15,sl,10.0,287.0
2014-08-16,sl,18.0,297.0
2014-08-17,sl,2.0,305.0
2014-08-18,sl,5.0,312.0
2014-08-19,sl,9.0,320.0
2014-08-20,sl,4.0,322.0
